# Example 06-02: Sampling from a Gamma(2, 1) target

In [ ]:
using Distributions
using Gadfly
using Lora

In [ ]:
plogtarget(p::Float64, v::Vector) = (v[1]-1)*log(p)-p/v[2]

In [ ]:
p = BasicContUnvParameter(:p, logtarget=plogtarget, nkeys=3)

In [ ]:
model = likelihood_model([Constant(:k), Constant(:θ), p], isindexed=false)

In [ ]:
psetproposal(x::Float64) = Truncated(Normal(x), 0, Inf)

In [ ]:
sampler = MH(psetproposal, symmetric=false)

In [ ]:
mcrange = BasicMCRange(nsteps=100000, burnin=10000)

In [ ]:
v0 = Dict(:k=>2., :θ=>1., :p=>2.)

In [ ]:
job = BasicMCJob(model, sampler, mcrange, v0)

In [ ]:
run(job)

In [ ]:
chain = output(job)

In [ ]:
mean(chain)

In [ ]:
acceptance(chain, diagnostics=false)

In [ ]:
xrange = 0.001:0.001:15
mcplot = plot(
  Guide.title("Gamma(2, 1)"),
  layer(
    x=collect(xrange),
    y=[pdf(Gamma(2, 1), v) for v in xrange],
    Geom.line,
    Theme(default_color=colorant"red")
  ),
  layer(x=chain.value, Geom.histogram(density=true))
)

In [ ]:
draw(PDF("/Users/theodore/tmp/gamma_hist.pdf", 8inch, 6inch), mcplot)